In [1]:
import re
import pandas as pd
import sys, os

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


from pyspark.sql.types import *

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf, concat_ws
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer

In [4]:
#описание и характеристики товаров магазина
data = spark.read.json('/labs/project02/item_details_full')
data=data.dropDuplicates().fillna(' ')                
data.registerTempTable('item_details_full')  

In [5]:
#!hadoop fs -cat /labs/project02/item_details_full | head -n 5

In [6]:
item = data.select(
       data.itemid.cast('integer'), 
       data.parent_id.cast('integer'), 
       concat_ws(' ', *['attr' + str(x) for x in range(78)]).alias('text'))

item.registerTempTable('item') 

In [7]:
item.take(2)

[Row(itemid=24489143, parent_id=None, text='Traite de la methode fumigatoire, ou, De l\'emploi medical des bains et douches de vapeurs. Воспроизведено в оригинальной авторской орфографии издания 1824 года (издательство "A Paris : Chez Gabon").   Traite de la methode fumigatoire, ou, De l\'emploi medical des bains et douches de vapeurs   Toussaint Rapou                                 Книга по Требованию                              Французский                                            978_5_8727_2685_2                                                  '),
 Row(itemid=23888213, parent_id=23496357, text='Современные кроссовки от Nike непременно станут вашими верными спутниками. Верх модели выполнен из натуральной кожи черного цвета и дополнен прострочкой. Боковые части модели дополнены вышивкой в виде логотипа бренда. Удобная шнуровка не только отлично смотрится, но и превосходно фиксирует модель на ноге. Внутри - отделка из текстиля. Плотная оригинальная подошва с амортизатором обеспечи

In [8]:
# id товара - id каталога
schema_t = StructType(fields=[StructField("itemid",StringType()),
                              StructField("catalogid",StringType())
                             ])
catalogs = spark.read.json("/labs/project02/catalogs",schema=schema_t)
catalogs.registerTempTable('catalogs')  

In [9]:
# товары + каталоги, к которым они относятся
item_cat = spark.sql('''SELECT trim(a.text) as attr,
                            a.itemid,
                            b.catalogid 
                     FROM item a 
                     JOIN catalogs b 
                       on a.itemid = b.itemid
                  ''')
item_cat.registerTempTable('item_cat') 

In [10]:
#Токенизация
@F.udf(ArrayType(StringType()))
def tokenizer(text):
    regex = re.compile(r'[\w\d]', re.U)
    text = text.lower()
    return regex.findall(text)

In [11]:
df_with_words_udf = item_cat.withColumn('words', tokenizer('attr'))

In [12]:
htf = HashingTF(inputCol="words",
                outputCol="TFFeatures")
feats = htf.transform(df_with_words_udf)

In [13]:
idf = IDF(inputCol="TFFeatures", outputCol="features")
idfModel = idf.fit(feats)
upd_df = idfModel.transform(feats)

In [14]:
from pyspark.ml.feature import Normalizer
t = Normalizer(inputCol='features', outputCol='norm_features', p=2.0)
upd_df_2 = t.transform(upd_df)

In [15]:
upd_df_2 = upd_df_2.drop('attr','words','TFFeatures', 'features').cache()   

In [16]:
upd_df_2.registerTempTable('feats')  

In [17]:
upd_df_2.take(1)

[Row(itemid=64423, catalogid='1133234', norm_features=SparseVector(262144, {10108: 0.0704, 15554: 0.0253, 17222: 0.0147, 24417: 0.0145, 34719: 0.0442, 34893: 0.0354, 47875: 0.2953, 52032: 0.5579, 58109: 0.0432, 62423: 0.2106, 66836: 0.1042, 67583: 0.2472, 70917: 0.1554, 80176: 0.2023, 95457: 0.0543, 105732: 0.1832, 105964: 0.0734, 107349: 0.0431, 109465: 0.0162, 113838: 0.0455, 118126: 0.0733, 132812: 0.0729, 145084: 0.1739, 147580: 0.1548, 147711: 0.2, 157485: 0.1697, 162209: 0.0269, 165164: 0.0624, 174135: 0.0621, 174776: 0.1688, 182498: 0.0088, 213268: 0.0284, 227410: 0.0143, 230215: 0.0251, 233878: 0.0259, 234682: 0.1052, 236968: 0.1605, 237265: 0.0351, 240533: 0.3894, 250208: 0.0167}))]

In [18]:
#ИНСАЙТ ОТ МОИХ КОЛЛЕГ, которые ходили на прошлый выпуск NPL
#они сказали, что такой подход сработал: 

'''
Искать похожие товары непосредственно в том же каталоге, где находится и основной товар. 
Сравнивать (краткое описание + основные характеристики) товаров косинусной мерой, 
где мера похожести будет соответствовать весу рекомендуемого товара
'''

'\nИскать похожие товары непосредственно в том же каталоге, где находится и основной товар. \nСравнивать (краткое описание + основные характеристики) товаров косинусной мерой, \nгде мера похожести будет соответствовать весу рекомендуемого товара\n'

In [19]:
#тестовые товары
test_schema = StructType(fields=[StructField("item",StringType())])
item_test = spark.read.json("/labs/project02/ozon_test.txt",schema=test_schema)
item_test.registerTempTable('test')  

In [20]:
#берем товары из теста
#catalogs - таблица из "/labs/project02/catalogs"
item_cat_test = spark.sql('select * from catalogs where itemid in (SELECT item FROM test)') 
item_cat_test.registerTempTable('item_cat_test') 

In [21]:
#берем уникальные id каталогов, чтобы далее по ним пройтись
cats = spark.sql('select distinct catalogid from item_cat_test ')
cats = cats.toPandas() 

In [22]:
cats_ids = [int(cat) for cat in cats['catalogid'].values]

In [ ]:
with open('project02.txt','w') as f:
    for i in tqdm(cats_ids):  #проходим по каталогам
        feats = spark.sql('select * from feats where catalogid={}'.format(i))            
        feats.cache()  
        feats.registerTempTable('feats_i') 
        short_item_cat_test = item_cat_test.filter('catalogid={}'.format(i)) #отберём товары теста текущей группы
        short_item_cat_test = short_item_cat_test.toPandas() 
        for i in tqdm(range(short_item_cat_test['itemid'].count())): 
            row = spark.sql('SELECT * FROM feats_i WHERE itemid={}'.format(short_item_cat_test['itemid'][i]))\
                                                                          .first()
            # для каждого row найдём косинусную меру товаров из его (товара из теста) группы
            #нужно получить resultData['itemid','similarity']
            
            pn={'item' : short_item_cat_test['itemid'][i],
                'recoms': {r.itemid : r.similarity for r in resultData[['itemid','similarity']].take(101)[0:100]}}
            f.write(str(pn).replace("'",'"')+'\n')